# Template Auditing Tool - TAT

The point of this project is to provide a tool for finding Unicode characters in templates. We can drop in a .csv and get a report of which templates have Unicode characters. This tool compares the length of a template message in characters vs. the length in bytes, as demonstrated in the code cell below. If character length and byte length differ, we know there are Unicode characters.

In [16]:
print(len('résumé')) # 6 characters
print(len('résumé'.encode('utf-8'))) # 8 bytes

6
8


2.0 templates are encoded as UTF-8.<br>
1.0 templates are ANSI.

Confirmed by opening the [.csv dumps](https://app.hubspot.com/contacts/21880972/record/0-5/1722173983/view/0?engagement=36665045903) in Notepad++, which displays the encoding on the far right of the bottom bar.

In [17]:
# We'll use pandas for data wrangling
import pandas as pd

# Replace the .csv below as needed
# Make sure to specify the correct encoding
# UTF-8 for 2.0 Templates
# ANSI for 1.0 Templates

templates = pd.read_csv("T2C1.0_TemplateMessagesReport_07.03.csv", encoding='ANSI')
# templates = pd.read_csv("2_0Templates.csv", encoding='UTF-8')

templates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165725 entries, 0 to 165724
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   CompanyName         165725 non-null  object
 1   TenantLevelOrgCode  165725 non-null  object
 2   LevelOneOrgName     165725 non-null  object
 3   LevelOneOrgCode     165725 non-null  object
 4   Org->Div->Group     165725 non-null  object
 5   GroupCode           165725 non-null  object
 6   TemplateID          165725 non-null  int64 
 7   TemplateName        165725 non-null  object
 8   TemplateFrom        165721 non-null  object
 9   TemplateMessage     165724 non-null  object
 10  TextMessage         165725 non-null  object
 11  TextMessageLength   165725 non-null  object
 12  TemplateType        165725 non-null  object
 13  Status              165723 non-null  object
 14  CreatedOn           165723 non-null  object
 15  ModifiedOn          45095 non-null   object
dtypes:

C:\Users\JarvisMartin\AppData\Local\Temp\ipykernel_8424\490742308.py:9: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  templates = pd.read_csv("T2C1.0_TemplateMessagesReport_07.03.csv", encoding='ANSI')


In [18]:
# View the first three rows
# to understand columns and their data
templates.head(3)

,CompanyName,TenantLevelOrgCode,LevelOneOrgName,LevelOneOrgCode,Org->Div->Group,GroupCode,TemplateID,TemplateName,TemplateFrom,TemplateMessage,TextMessage,TextMessageLength,TemplateType,Status,CreatedOn,ModifiedOn
0,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,10,Generic Past Due - Spanish,1803 Capital:,Necesitamos hablar con usted sobre su cuenta. ...,1803 Capital: Necesitamos hablar con usted sob...,127,MC,Active,11/2/22 10:47 AM,NaN
1,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,11,Online Payments - Spanish,1803 Capital:,Buenas noticias! Ahora puede pagar los pagos d...,1803 Capital: Buenas noticias! Ahora puede pag...,159,MC,Active,11/2/22 10:47 AM,NaN
2,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,12,Past Due: 10-Days - Spanish,1803 Capital:,Tu pago se vencio hace 10 dias! Queremos ayuda...,1803 Capital: Tu pago se vencio hace 10 dias! ...,155,MC,Active,11/2/22 10:47 AM,NaN


In [24]:
# Check the output above, but as of this writing,
# 2.0 template messages are in the "Message" column
# 1.0 templates messages are in the "TextMessage" column
if 'Message' in templates.columns:
    # messages = templates["Message"] # 2.0 Templates
    templates["Msg_Characters"] = templates.Message.str.len()
    templates["Msg_Bytes"] = templates["Message"].apply(lambda x: len(x.encode('utf-8')))
else:
    # messages = templates.iloc[:, 6]
    # messages = templates["TextMessage"]

    # For consistency, put message body in Message column
    templates["Message"] = templates["TextMessage"]

    templates["Msg_Characters"] = templates.TextMessage.str.len()
    templates["Msg_Bytes"] = templates["TextMessage"].apply(lambda x: len(x.encode('utf-8')))

templates.head(3)

,CompanyName,TenantLevelOrgCode,LevelOneOrgName,LevelOneOrgCode,Org->Div->Group,GroupCode,TemplateID,TemplateName,TemplateFrom,TemplateMessage,TextMessage,TextMessageLength,TemplateType,Status,CreatedOn,ModifiedOn,Msg_Characters,Msg_Bytes,Message
0,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,10,Generic Past Due - Spanish,1803 Capital:,Necesitamos hablar con usted sobre su cuenta. ...,1803 Capital: Necesitamos hablar con usted sob...,127,MC,Active,11/2/22 10:47 AM,NaN,127,127,1803 Capital: Necesitamos hablar con usted sob...
1,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,11,Online Payments - Spanish,1803 Capital:,Buenas noticias! Ahora puede pagar los pagos d...,1803 Capital: Buenas noticias! Ahora puede pag...,159,MC,Active,11/2/22 10:47 AM,NaN,159,159,1803 Capital: Buenas noticias! Ahora puede pag...
2,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,12,Past Due: 10-Days - Spanish,1803 Capital:,Tu pago se vencio hace 10 dias! Queremos ayuda...,1803 Capital: Tu pago se vencio hace 10 dias! ...,155,MC,Active,11/2/22 10:47 AM,NaN,155,155,1803 Capital: Tu pago se vencio hace 10 dias! ...


In [93]:
unicode_detected = templates.query('Msg_Characters != Msg_Bytes')

unicode_detected

,CompanyName,TenantLevelOrgCode,LevelOneOrgName,LevelOneOrgCode,Org->Div->Group,GroupCode,TemplateID,TemplateName,TemplateFrom,TemplateMessage,TextMessage,TextMessageLength,TemplateType,Status,CreatedOn,ModifiedOn,Msg_Characters,Msg_Bytes,Message
194,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division->60s,7180gl9,2438,48-hour Text Reminder: (Spanish),Su oferta,para reducir sus pagos de préstamo y poner su ...,Su oferta para reducir sus pagos de préstamo y...,264,MC,Active,5/18/23 9:12 PM,NaN,264,268,Su oferta para reducir sus pagos de préstamo y...
243,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division->90s,d16p2q6,2467,48-hour Text Reminder: (Spanish),Su oferta,para reducir sus pagos de préstamo y poner su ...,Su oferta para reducir sus pagos de préstamo y...,264,MC,Active,5/18/23 9:13 PM,NaN,264,268,Su oferta para reducir sus pagos de préstamo y...
706,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division - CCB->CC...,5w25073,2155,48-hour Text Reminder: (Spanish),Su oferta,para reducir sus pagos de préstamo y poner su ...,Su oferta para reducir sus pagos de préstamo y...,265,MC,Active,9/15/22 3:40 PM,9/27/22 5:06 PM,265,269,Su oferta para reducir sus pagos de préstamo y...
708,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division - CCB->CC...,5w25073,2157,Loan Modification Text: (Spanish),Usted podra,calificar para una modificación del préstamo. ...,Usted podra calificar para una modificación de...,224,MC,Active,9/15/22 3:40 PM,NaN,224,227,Usted podra calificar para una modificación de...
1089,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division->Account ...,v6u8457,2161,Loan Modification Text: (Spanish),Usted podra,calificar para una modificación del préstamo. ...,Usted podra calificar para una modificación de...,224,MC,Active,9/20/22 4:39 PM,NaN,224,227,Usted podra calificar para una modificación de...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165502,WHFDI dba Farmers Home Furniture,j39cm89,Farmers Home Furniture,8hw8b37,Farmers Home Furniture->Account Division->TwoW...,i98ka61,69,1-Way - Spanish - PTP Confirmation 2,Farmers Home Furniture:,Gracias por su respuesta. Hemos hecho una nota...,Farmers Home Furniture: Gracias por su respues...,150,MC,Active,5/25/21 12:08 AM,NaN,150,151,Farmers Home Furniture: Gracias por su respues...
165631,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,e03cr16,80,CLN - Status - Approved Contact,Georgia United:,"Congratulations, your loan is approved! Please...","Georgia United: Congratulations, your loan is ...",153,MC,Active,12/15/22 5:35 PM,12/19/22 3:23 PM,153,154,"Georgia United: Congratulations, your loan is ..."
165673,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,31m0e79,45,MLN - Hours,Georgia United:,We’re available Monday – Friday from 8:30am – ...,Georgia United: We’re available Monday – Frida...,130,MC,Active,12/15/22 5:35 PM,12/19/22 3:24 PM,130,138,Georgia United: We’re available Monday – Frida...
165683,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,31m0e79,55,MLN - Status - Approved Contact,Georgia United:,"Congratulations, your loan is approved! Please...","Georgia United: Congratulations, your loan is ...",153,MC,Active,12/15/22 5:35 PM,12/19/22 3:24 PM,153,154,"Georgia United: Congratulations, your loan is ..."


In [113]:
# GSM 7 Char Set
# These count as one character
gsm_char_set = {'@', 'Δ', ' ', '0', '¡', 'P', '¿', 'p', '£', '_', '!', '1', 'A', 'Q', 'a', 'q', '$', 'Φ', '"', '2', 'B', 'R', 'b', 'r', '¥', 'Γ', '#', '3', 'C', 'S', 'c', 's', 'è', 'Λ', '¤', '4', 'D', 'T', 'd', 't', 'é', 'Ω', '%', '5', 'E', 'U', 'e', 'u', 'ù', 'Π', '&', '6', 'F', 'V', 'f', 'v', 'ì', 'Ψ', '‘', '7', 'G', 'W', 'g', 'w', 'ò', 'Σ', '(', '8', 'H', 'X', 'h', 'x', 'Ç', 'Θ', ')', '9', 'I', 'Y', 'i', 'y', 'LF', 'Ξ', '*', ':', 'J', 'Z', 'j', 'z', 'Ø', 'ESC', '+', ';', 'K', 'Ä', 'k', 'ä', 'ø', 'Æ', ',', '<', 'L', 'Ö', 'l', 'ö', 'CR', 'æ', '-', '=', 'M', 'Ñ', 'm', 'ñ', 'Å', 'ß', '.', '>', 'N', 'Ü', 'n', 'ü', 'å', 'É', '/', '?', 'O', '§', 'o', 'à'}

# GSM 7 Extended Set
# these require escape (\) and count as two characters
gsm_extended_set = {'^', '€', '|', '{', '}', '[', ']', '~'}

# gsm_set = gsm_char_set.union(gsm_extended_set)

# gsm_set_str = "@ £ $ ¥ è é ù ì ò Ç  Ø ø  Å å Δ _ Φ Γ Λ Ω Π Ψ Σ Θ Ξ   ^ { } \ [ ~ ] | € Æ æ ß É  ! \" # ¤ % & ' ( ) * + , - . / 0 1 2 3 4 5 6 7 8 9 : ; < = > ? ¡ A B C D E F G H I J K L M N O P Q R S T U V W X Y Z Ä Ö Ñ Ü § ¿ a b c d e f g h i j k l m n o p q r s t u v w x y z ä ö ñ ü à"
# gsm_list = gsm_set_str.split(' ')
# # print(gsm_list)
# gsm_set = set(gsm_list).union({" "})
gsm_set = {'t', '%', 'W', 'L', 'Q', '¥', ')', 'N', 'ß', 'o', '&', 'c', '4', 'I', 'é', 'Φ', 'n', 'G', 'Π', '€', 'X', '>', 'f', 'à', 's', 'm', 'l', '<', 'd', 'O', 'p', '#', '~', 'ñ', 'Δ', 'M', 'x', '*', '$', 'Ü', '9', 'P', 'Ø', 'Σ', 'Ξ', 'E', 'i', 'Ψ', '1', ',', 'ì', '(', 'Θ', 'Λ', 'F', '_', '@', '/', 'h', 'V', 'ä', '+', ']', 'z', 'a', 'D', 'T', '"', 'e', 'É', ':', '!', 'k', 'w', 'ù', 'Ä', 'g', '£', '0', '6', '§', 'j', 'Y', 'H', ' ', '|', '¤', '-', 'Æ', 'J', 'y', '{', '?', '^', '2', 'S', '¿', ';', '=', 'ö', 'K', 'b', '5', '.', 'å', 'ü', '7', 'æ', 'U', 'Å', '\\', 'Γ', 'ò', 'Ω', '¡', 'u', 'ø', "'", '}', 'B', 'r', 'Z', 'è', '3', 'Ö', 'A', '8', 'Ç', 'v', 'q', '[', 'Ñ', 'R', 'C'}

print(gsm_set)
print(type(gsm_set))

def findNonGsm(str):
    # Convert String to Set
    str_set = set(str)

    diff = str_set.difference(gsm_set)

    if len(diff) == 0:
        return "NONE"
    else:
        return diff

# print(findNonGsm('Please reply STOP to this message to prevent future texts. We apologize for the wrong number.'))

{'t', '%', 'W', 'L', 'Q', '¥', ')', 'N', 'ß', 'o', '&', 'c', '4', 'I', 'é', 'Φ', 'n', 'G', 'Π', '€', 'X', '>', 'f', 'à', 's', 'm', 'l', '<', 'd', 'O', 'p', '#', '~', 'ñ', 'Δ', 'M', 'x', '*', '$', 'Ü', '9', 'P', 'Ø', 'Σ', 'Ξ', 'E', 'i', 'Ψ', '1', ',', 'ì', '(', 'Θ', 'Λ', 'F', '_', '@', '/', 'h', 'V', 'ä', '+', ']', 'z', 'a', 'D', 'T', '"', 'e', 'É', ':', '!', 'k', 'w', 'ù', 'Ä', 'g', '£', '0', '6', '§', 'j', 'Y', 'H', ' ', '|', '¤', '-', 'Æ', 'J', 'y', '{', '?', '^', '2', 'S', '¿', ';', '=', 'ö', 'K', 'b', '5', '.', 'å', 'ü', '7', 'æ', 'U', 'Å', '\\', 'Γ', 'ò', 'Ω', '¡', 'u', 'ø', "'", '}', 'B', 'r', 'Z', 'è', '3', 'Ö', 'A', '8', 'Ç', 'v', 'q', '[', 'Ñ', 'R', 'C'}
<class 'set'>


In [114]:
templates["NonGSM_Chars"] = templates["Message"].apply(lambda x: findNonGsm(x))

templates

,CompanyName,TenantLevelOrgCode,LevelOneOrgName,LevelOneOrgCode,Org->Div->Group,GroupCode,TemplateID,TemplateName,TemplateFrom,TemplateMessage,TextMessage,TextMessageLength,TemplateType,Status,CreatedOn,ModifiedOn,Msg_Characters,Msg_Bytes,Message,NonGSM_Chars
0,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,10,Generic Past Due - Spanish,1803 Capital:,Necesitamos hablar con usted sobre su cuenta. ...,1803 Capital: Necesitamos hablar con usted sob...,127,MC,Active,11/2/22 10:47 AM,NaN,127,127,1803 Capital: Necesitamos hablar con usted sob...,NONE
1,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,11,Online Payments - Spanish,1803 Capital:,Buenas noticias! Ahora puede pagar los pagos d...,1803 Capital: Buenas noticias! Ahora puede pag...,159,MC,Active,11/2/22 10:47 AM,NaN,159,159,1803 Capital: Buenas noticias! Ahora puede pag...,NONE
2,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,12,Past Due: 10-Days - Spanish,1803 Capital:,Tu pago se vencio hace 10 dias! Queremos ayuda...,1803 Capital: Tu pago se vencio hace 10 dias! ...,155,MC,Active,11/2/22 10:47 AM,NaN,155,155,1803 Capital: Tu pago se vencio hace 10 dias! ...,NONE
3,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,13,Past Due: 17-Days - Spanish,1803 Capital:,Queremos ayudar. Su pago se vencio el {MISC1}....,1803 Capital: Queremos ayudar. Su pago se venc...,141,MC,Active,11/2/22 10:47 AM,NaN,141,141,1803 Capital: Queremos ayudar. Su pago se venc...,NONE
4,1803 Capital,ml03l88,1803 Capital - Spanish,3u591t4,1803 Capital - Spanish->1803 Capital Account D...,80q5hr1,14,Past Due: 3-Days - Spanish,1803 Capital:,Su pago se vencio hace 3 dias. Para actualizar...,1803 Capital: Su pago se vencio hace 3 dias. P...,141,MC,Active,11/2/22 10:47 AM,NaN,141,141,1803 Capital: Su pago se vencio hace 3 dias. P...,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165720,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,e03cr16,72,CLN - Processing - eDoc incomplete,Georgia United:,We noticed that you haven't signed all eDocs. ...,Georgia United: We noticed that you haven't si...,159,MC,Active,12/15/22 5:35 PM,12/19/22 3:23 PM,159,159,Georgia United: We noticed that you haven't si...,NONE
165721,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,e03cr16,73,CLN - Processing - eDoc not signed,Georgia United:,We noticed that you haven't signed your eDocs....,Georgia United: We noticed that you haven't si...,160,MC,Active,12/15/22 5:35 PM,12/19/22 3:23 PM,160,160,Georgia United: We noticed that you haven't si...,NONE
165722,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,e03cr16,74,CLN - Processing - Loan App Approved,Georgia United:,Congrats! Your loan has been approved! Call us...,Georgia United: Congrats! Your loan has been a...,157,MC,Active,12/15/22 5:35 PM,12/19/22 3:23 PM,157,157,Georgia United: Congrats! Your loan has been a...,NONE
165723,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,e03cr16,75,CLN - Processing - Loan Funded,Georgia United:,Thanks for your business! We look forward to h...,Georgia United: Thanks for your business! We l...,160,MC,Active,12/15/22 5:35 PM,12/19/22 3:23 PM,160,160,Georgia United: Thanks for your business! We l...,NONE


In [115]:
nonGSM_detected = templates[templates["NonGSM_Chars"] != "NONE"]

nonGSM_detected

,CompanyName,TenantLevelOrgCode,LevelOneOrgName,LevelOneOrgCode,Org->Div->Group,GroupCode,TemplateID,TemplateName,TemplateFrom,TemplateMessage,TextMessage,TextMessageLength,TemplateType,Status,CreatedOn,ModifiedOn,Msg_Characters,Msg_Bytes,Message,NonGSM_Chars
194,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division->60s,7180gl9,2438,48-hour Text Reminder: (Spanish),Su oferta,para reducir sus pagos de préstamo y poner su ...,Su oferta para reducir sus pagos de préstamo y...,264,MC,Active,5/18/23 9:12 PM,NaN,264,268,Su oferta para reducir sus pagos de préstamo y...,"{ó, á, í}"
243,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division->90s,d16p2q6,2467,48-hour Text Reminder: (Spanish),Su oferta,para reducir sus pagos de préstamo y poner su ...,Su oferta para reducir sus pagos de préstamo y...,264,MC,Active,5/18/23 9:13 PM,NaN,264,268,Su oferta para reducir sus pagos de préstamo y...,"{ó, á, í}"
706,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division - CCB->CC...,5w25073,2155,48-hour Text Reminder: (Spanish),Su oferta,para reducir sus pagos de préstamo y poner su ...,Su oferta para reducir sus pagos de préstamo y...,265,MC,Active,9/15/22 3:40 PM,9/27/22 5:06 PM,265,269,Su oferta para reducir sus pagos de préstamo y...,"{ó, á, í}"
708,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division - CCB->CC...,5w25073,2157,Loan Modification Text: (Spanish),Usted podra,calificar para una modificación del préstamo. ...,Usted podra calificar para una modificación de...,224,MC,Active,9/15/22 3:40 PM,NaN,224,227,Usted podra calificar para una modificación de...,{ó}
1089,800 Loan Mart,45p09,800 LoanMart Org,57x48,800 LoanMart Org->Acct Mgmt Division->Account ...,v6u8457,2161,Loan Modification Text: (Spanish),Usted podra,calificar para una modificación del préstamo. ...,Usted podra calificar para una modificación de...,224,MC,Active,9/20/22 4:39 PM,NaN,224,227,Usted podra calificar para una modificación de...,{ó}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165502,WHFDI dba Farmers Home Furniture,j39cm89,Farmers Home Furniture,8hw8b37,Farmers Home Furniture->Account Division->TwoW...,i98ka61,69,1-Way - Spanish - PTP Confirmation 2,Farmers Home Furniture:,Gracias por su respuesta. Hemos hecho una nota...,Farmers Home Furniture: Gracias por su respues...,150,MC,Active,5/25/21 12:08 AM,NaN,150,151,Farmers Home Furniture: Gracias por su respues...,{í}
165631,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,e03cr16,80,CLN - Status - Approved Contact,Georgia United:,"Congratulations, your loan is approved! Please...","Georgia United: Congratulations, your loan is ...",153,MC,Active,12/15/22 5:35 PM,12/19/22 3:23 PM,153,154,"Georgia United: Congratulations, your loan is ...",{ }
165673,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,31m0e79,45,MLN - Hours,Georgia United:,We’re available Monday – Friday from 8:30am – ...,Georgia United: We’re available Monday – Frida...,130,MC,Active,12/15/22 5:35 PM,12/19/22 3:24 PM,130,138,Georgia United: We’re available Monday – Frida...,"{–, ’}"
165683,zMIGRATED - Georgia United Credit Union,y5jg462,Georgia United Credit Union,9t42p17,Georgia United Credit Union->Account Division-...,31m0e79,55,MLN - Status - Approved Contact,Georgia United:,"Congratulations, your loan is approved! Please...","Georgia United: Congratulations, your loan is ...",153,MC,Active,12/15/22 5:35 PM,12/19/22 3:24 PM,153,154,"Georgia United: Congratulations, your loan is ...",{ }


Add SMS parts (like sms calculator tool)<br>
Have report of just multiple part (not unicode)

Identify most at risk customers. Which customers send the most messages? Ask Thanoj? 

Separate thread: look into Superset & Quicksite / reporting. Per Subra, reach out to Shuaib.
ask Thanoj about Quicksight

Build 1 report with Quicksite and one with Superset